In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

## RAG 절차
- 준비 : https://law.go.kr/법령/소득세법 에서 doc다운로드(hwp는 파이썬 못 읽음. pdf는 한글의 경우 단어가 짤림) 받아 파일형식을 docx로 변경

1. 문서를 읽는다(python-docx이용) : \n은 제외
2. 읽어온 문서를 쪼갠다(tiktoken 이용)
    * 모델의 context window 확인
    * chunk(문서를 쪼갠 하나)가 길면 비용과 시간이 많이 들어
3. 쪼갠 문서를 임베딩 -> vector database에 저장 -> chroma(local vector database==vector store)/pinecorn(클라우드 vector database=vector store)
4. 질문과 vector DB의 유사도 검색                                              
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달하여 답변 생성


# 1. 문서를 읽는다(python-docx이용)
- pip install python-docx
- pip install langchain-community

In [2]:
%pip install -U -q langchain-community docx2txt python-docx

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 이렇게 docx파일을 읽어오면 하나로 한꺼번에 가져옴(\n포함)
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('data/소득세법(법률)(제21065호)(20260102).docx')
document = loader.load() #한꺼번에 load
#document

ValueError: File path data/소득세법(법률)(제21065호)(20260102).docx is not a valid file or url

In [ ]:
from docx import Document
document = Document('./data/소득세법(법률)(제21065호)(20260102).docx')
print(document)
print(dir(document)) # 객체가 가지고 있는 속성이름

In [ ]:
print(len(document.paragraphs)) # \n마다 새로운 paragraph로 생성
for paragraph in document.paragraphs[100:105]:
    #print(dir(paragraph))
    print(paragraph.text)

In [ ]:
# \n이 제거된 문서 전체 내용을 full_text
full_text = ""
for paragraph in document.paragraphs:
    full_text += paragraph.text + " "
full_text

In [ ]:
# docx 문서의 글자수 및 paragraph 수
print(f'글자수 : {len(full_text)}글자')
print('paragraph수 :', len(document.paragraphs))

# 2. 문서를 쪼갠다
- pip install tiktoken : OpenAI의 공식 토크나이저 라이브러리
- full_text -> 토큰 단위로 쪼개서 숫자
- 1500(?)토큰씩 문서를 쪼개기

In [ ]:
%pip install -q tiktoken

In [ ]:
import tiktoken
# tiktoken이 인식가능한 모델 만 가능 : nano모델은 불가. gpt-4x, gpt-3.5-turbo,...
encoder = tiktoken.encoding_for_model("gpt-4o-mini")
# 문자들 -> 숫자 리스트
encoding = encoder.encode(full_text)
# 숫자리스트 -> 문자들
decoded = encoder.decode(encoding)

In [ ]:
print(f'글자수 : {len(full_text)}글자')
print('full_text의 전체 토큰수 :', len(encoding))
print('문자들 :', decoded[:10])
print('숫자들 :', encoding[:10])

In [ ]:
print(encoder.encode("소득세법 어쩌구 저쩌구"))

In [ ]:
encoder.encode("소득세"), encoder.encode(" 친구"), encoder.encode("Hello"), encoder.encode("안녕")

In [ ]:
print(list(range(0, 161977, 1500)))

In [ ]:
# full_text를 쪼개는 함수(chunk_size단위로) : chunk_list반환
import tiktoken
def split_text(full_text=full_text, chunk_size=1500):
    encoder = tiktoken.encoding_for_model("gpt-4o-mini")
    total_encoding = encoder.encode(full_text) # 문자 full_text를 토큰단위 숫자 list 
    total_token_count = len(total_encoding) # 전체 토큰 수
    chunk_list = []
    for i in range(0, total_token_count, chunk_size):
        # print(f"{i} / {total_token_count}") # i/전체토큰수
        chunk_encoding = total_encoding[i : i+chunk_size] # chunk_size만큼 분할된 숫자 list
        chunk_decoded = encoder.decode(chunk_encoding)
        # print(chunk_encoding, '=>', chunk_decoded)
        chunk_list.append(chunk_decoded)
    return chunk_list

In [ ]:
example_text = "소득세법 법률 일부 개정함 이자소득 배당소득"
split_text(example_text, 5)

In [ ]:
chunk_list = split_text(full_text, 1500)

In [ ]:
len(chunk_list) # chunk 수

# 3. 쪼갠 문서를 임베딩 -> vector database에 저장
- pip install chromadb

In [ ]:
%pip install -q chromadb --no-warn-script-location

In [ ]:
import chromadb
chroma_client = chromadb.Client()

In [ ]:
# collection은 RDB의 테이블 개념
collection_name = "tax_collection"

In [ ]:
# 임베딩 객체
from dotenv import load_dotenv
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
import os
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
openai_embedding = OpenAIEmbeddingFunction(
    api_key=openai_key,
    model_name="text-embedding-3-large"
)

In [ ]:
# collection에 입력할 때 사용할 id들 : 문자
ids = [str(id) for id in range(len(chunk_list))]
print(ids)

In [ ]:
# chroma db에 collection 생성
tax_collection = chroma_client.get_or_create_collection(
    name=collection_name,
    embedding_function=openai_embedding
)

In [ ]:
tax_collection.add(documents=chunk_list, ids=ids)

In [ ]:
# tax_collection 가져오기
results = tax_collection.get(include=['embeddings', 'documents', 'metadatas'])
print("전체 chunk 수 :", len(results['ids']))
print("ID 목록(처음5개만) :", results['ids'][:5])
print("문서내용(0번째 100글자) :", results['documents'][10][:100])
print("임베딩 vector(0번째) :", results['embeddings'][0])
print("메타데이터(처음 5개만) :", results['metadatas'][:5])

# 4. 질문과 vector Database의 유사도 검색

In [ ]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_doc = tax_collection.query(query_texts=query,
                                    n_results=3 # 유사도가 높은 3개 추출(기본값은 10)
                                    )

In [ ]:
retrieved_doc['documents'][0]

# 5. 유사도 검색으로 나온 문서를 LLM에 질문과 같이 전달
- retrieved_doc['documents'][0]

In [ ]:
from openai import OpenAI
load_dotenv()
client = OpenAI()
response = client.chat.completions.create(
    model = "gpt-4o-mini", # tiktoken.encoding_for_model()에서 쓴 모델 사용
    messages=[
        {
            "role":"system", 
            "content":f"""당신은 한국 소득세 전문가입니다. 
아래의 내용을 참고해서 사용자 질문에 답변해 주세요
{retrieved_doc['documents'][0]}"""
        },
        {"role":"user", "content":query}
    ]
)

In [ ]:
print(response.choices[0].message.content)